In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from rlglue import rl_glue as RLGlue
from agents import QLearningAgent # import agent
from environments import TwoChoiceMDP # import environment

In [2]:
np.random.seed(0)

agents = {
    "Q-learning": QLearningAgent,
}
env = TwoChoiceMDP
all_reward_sums = {} # Contains sum of rewards during episode
all_state_visits = {} # Contains state visit counts during the last 10 episodes
agent_info = {"num_actions": 2, "num_states": 9, "epsilon": 0.1, "step_size": 0.5, "discount": 0.5}
env_info = {}
num_runs = 100 # The number of runs
num_episodes = 100 # The number of episodes in each run

for algorithm in ["Q-learning"]:
    all_reward_sums[algorithm] = []
    all_state_visits[algorithm] = []
    for run in tqdm(range(num_runs)):
        agent_info["seed"] = run
        rl_glue1 = RLGlue.RLGlue(env, agents[algorithm])
        rl_glue1.rl_init(agent_info, env_info)

        reward_sums = []
        state_visits = np.zeros(8)
        for episode in range(num_episodes):
            if episode < num_episodes - 10:
                # Runs an episode
                rl_glue1.rl_episode(10000) 
            else: 
                # Runs an episode while keeping track of visited states
                state, action = rl_glue.rl_start()
                state_visits[state] += 1
                is_terminal = False
                while not is_terminal:
                    reward, state, action, is_terminal = rl_glue.rl_step()
                    state_visits[state] += 1
                
            reward_sums.append(rl_glue.rl_return())
            last_episode_total_reward = rl_glue.rl_return()
            
        all_reward_sums[algorithm].append(reward_sums)
        all_state_visits[algorithm].append(state_visits)

# plot results
for algorithm in ["Q-learning", "Expected Sarsa"]:
    plt.plot(np.mean(all_reward_sums[algorithm], axis=0), label=algorithm)
plt.xlabel("Episodes")
plt.ylabel("Sum of\n rewards\n during\n episode",rotation=0, labelpad=40)
plt.xlim(0,100)
plt.ylim(-100,0)
plt.legend()
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: arrays used as indices must be of integer (or boolean) type